# **Clustering Service numérique établissement**


Nous allons encore une fois procéder à un clustering pour réduire le nombre de colonnes suivantes en une seule colonne avec plusieurs modalités :
- *`'ServInt_NoteElev'`* : Les notes des élèves sont l'un des services de l'établissement accessibles hors établissement via internet
- *`'ServInt_AbsElev'`* : Les absences des élèves sont l'un des services de l'établissement accessibles hors établissement via internet
- *`'ServInt_EdtElevCls'`* : L'emploi du temps des élèves ou de la classe est l'un des services de l'établissement accessible hors
établissement via internet
- *`'ServInt_CahierTxt'`* : Le cahier de textes de la classe est l'un des services de l'établissement accessibles hors établissement
via internet
- *`'ServInt_DocRessPeda'`* : Les documents et ressources pédagogiques sont l'un des services de l'établissement accessibles hors établissement via internet
- *`'ServInt_AgdActuEtab'`* : L'agenda/Actualités de l'école est un service disponible par internet pour l'école
- *`'ServInt_PlatApp'`* : La plateforme d'apprentissage (Moodle, Claroline, MOOC, etc.) est l'un des services de l'établissement
accessible hors établissement via internet
- *`'ServInt_Autres'`* : D'autres services de l'établissement sont accessibles hors établissement via internet
- *`'ServInt_aucun'`* : Aucun service n'est proposé par l'école



Nous faisons comme nous avons pu le faire pour les analyses précédentes. Nous traitons ces colonnes ensemble car elles sont toutes liées car elles traitent des services que proposent un établissement au format numérique. Des informations qu'on peut accéder depuis Internet

### *Import des bibliothèques*

In [12]:
import pandas as pd
import prince as pc
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import plotly_express as px
import plotly.graph_objects as go

### *Import des données*

In [2]:
df = pd.read_csv('../data/lycee-college/fr-en-etic_2d.csv', sep=';')

#On consulte les colonnes existantes
df.columns

Index(['Code_UAI', 'Millesime', 'Code_nature', 'nature_uai', 'typ_etab',
       'Academie', 'Departement', 'NbEleve', 'NbEnseignant',
       'SiEtabCentreRessource', 'SiProjetNumEcole', 'SiEntDisponible',
       'SiProjEtabIntegreENT', 'Maint_PersCollect', 'Maint_PrestaExt',
       'Maint_PersonnelEducNatHsEtab', 'Maint_PersonnelEtab',
       'Maint_AutreNeSaitPas', 'Maint_Personne', 'NbRessourceEdit',
       'TpRess_ManuelNum', 'TpRess_AnimScienLogiSimu', 'TpRess_Bdd',
       'TpRess_LogiOutils', 'TpRess_OuvRef', 'TpRess_ResEntrainement',
       'TpRess_Autres', 'TpRess_aucune', 'ServInt_NoteElev', 'ServInt_AbsElev',
       'ServInt_EdtElevCls', 'ServInt_CahierTxt', 'ServInt_DocRessPeda',
       'ServInt_AgdActuEtab', 'ServInt_PlatApp', 'ServInt_Autres',
       'ServInt_aucun', 'NbTerminaux', 'NbTablette', 'NbTablettePC',
       'NbMicroMoins5', 'NbMicroPortable', 'NbPortAffEl', 'NbPortAffEns',
       'NbEleveEqASH', 'NbPosteEqASH', 'NbTBI', 'propClassesAvecTBI',
       'NbVideoProj',

### *Traitement des données*

On va changer d'index en affectant le code UAI car il est unique et il sera plus simple d'identifier les établissements par la suite. On va supprimer toutes les colonnes qui nous intéressenet pas et on va supprimer les lignes contenant des NaN

In [3]:
df_services = df.set_index("Code_UAI")

#On supprime les colonnes inutiles
df_services.drop(columns=['Millesime', 'Code_nature', 'nature_uai', 'typ_etab',
       'Academie', 'Departement', 'NbEleve', 'NbEnseignant',
       'SiEtabCentreRessource', 'SiProjetNumEcole', 'SiEntDisponible',
       'SiProjEtabIntegreENT', 'Maint_PersCollect', 'Maint_PrestaExt',
       'Maint_PersonnelEducNatHsEtab', 'Maint_PersonnelEtab',
       'Maint_AutreNeSaitPas', 'Maint_Personne', 'NbRessourceEdit',
       'TpRess_ManuelNum', 'TpRess_AnimScienLogiSimu', 'TpRess_Bdd',
       'TpRess_LogiOutils', 'TpRess_OuvRef', 'TpRess_ResEntrainement',
       'TpRess_Autres', 'TpRess_aucune', 'NbTerminaux', 'NbTablette', 'NbTablettePC',
       'NbMicroMoins5', 'NbMicroPortable', 'NbPortAffEl', 'NbPortAffEns',
       'NbEleveEqASH', 'NbPosteEqASH', 'NbTBI', 'propClassesAvecTBI',
       'NbVideoProj', 'NbClassMobile', 'NbLecteurMpx', 'NbImpr3D',
       'AccWeb_RTC', 'AccWeb_CableFibreOptique', 'AccWeb_ADSL',
       'AccWeb_AutresHautDebit', 'AccWeb_NeSaitPas', 'AccWeb_Aucun',
       'DebitWeb', 'SiWifi', 'SalleInternet', 'PostesInfoElvHorsCours',
       'SiPareFeuEtab', 'SiOuifiltrageWeb', 'ControlePosteriori',
       'SiCharteUsageWeb', 'Diffch_AnnexeeRI',
       'Diffch_DossierRentreeEnseignants', 'Diffch_CRConseilAdmin',
       'Diffch_DiffusionParents', 'Diffch_Autres', 'AccesParentCharte',
       'ElvAuthentif', 'SiVisioConferenc', 'SiEntUtilise',
       'TypeMatHandi_Tablette', 'TypeMatHandi_OrdiPort', 'TypeMatHandi_LogApp',
       'TypeMatHandi_Autre', 'Code_region', 'Libelle_region'], inplace=True)

#On supprime les lignes avec des valeurs manquantes
df_services.dropna(inplace=True)

On vérifie que les données indésirables ont bien été supprimées

In [7]:
df_services.head(3)

,ServInt_NoteElev,ServInt_AbsElev,ServInt_EdtElevCls,ServInt_CahierTxt,ServInt_DocRessPeda,ServInt_AgdActuEtab,ServInt_PlatApp,ServInt_Autres,ServInt_aucun
Code_UAI,,,,,,,,,
0810016C,oui,oui,oui,oui,oui,oui,non,oui,non
0810026N,oui,oui,oui,oui,oui,oui,non,non,non
0810041E,oui,oui,oui,oui,oui,oui,non,oui,non


In [8]:
for column in df_services.columns:
    print("Nombre de valeurs manquantes dans la colonne ", column, ":", df_services[column].isna().sum())

Nombre de valeurs manquantes dans la colonne  ServInt_NoteElev : 0
Nombre de valeurs manquantes dans la colonne  ServInt_AbsElev : 0
Nombre de valeurs manquantes dans la colonne  ServInt_EdtElevCls : 0
Nombre de valeurs manquantes dans la colonne  ServInt_CahierTxt : 0
Nombre de valeurs manquantes dans la colonne  ServInt_DocRessPeda : 0
Nombre de valeurs manquantes dans la colonne  ServInt_AgdActuEtab : 0
Nombre de valeurs manquantes dans la colonne  ServInt_PlatApp : 0
Nombre de valeurs manquantes dans la colonne  ServInt_Autres : 0
Nombre de valeurs manquantes dans la colonne  ServInt_aucun : 0


On va ensuite renommer les colonnes comme le suffixe `ServInt` se répète

In [10]:
old_columns = df_services.columns

for column in old_columns:
    df_services.rename(columns={column: column.replace("ServInt_", "").lower()}, inplace=True)

df_services.columns

Index(['noteelev', 'abselev', 'edtelevcls', 'cahiertxt', 'docresspeda',
       'agdactuetab', 'platapp', 'autres', 'aucun'],
      dtype='object')

On va maintenant transfomer les "oui" et "non" en valeurs binaires pour pouvoir faire une classification non supervisée. On va transformer les "oui" en 1 et les "non" en 0

In [11]:
for column in df_services.columns:
    df_services[column] = df_services[column].replace({'oui': 1, 'non': 0})
    df_services[column] = df_services[column].astype('float64')

df_services.head(3)

,noteelev,abselev,edtelevcls,cahiertxt,docresspeda,agdactuetab,platapp,autres,aucun
Code_UAI,,,,,,,,,
0810016C,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0
0810026N,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
0810041E,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0


### Corrélation

On va d'abord voir si une corrélation existe entre ces différentes variables.

In [13]:
columns_names = df_services.columns

heat = go.Heatmap(
    z = df_services.corr(),
    x = columns_names,
    y = columns_names,
    xgap=1,
    ygap=1,
    colorbar_thickness=20,
    colorbar_ticklen=3,
    hovertext= df_services.corr(),
    hoverinfo='text'
)

layout = go.Layout(
    title='Matrice de corrélation',
    width=600,
    height=600,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    yaxis_autorange='reversed'
)

fig = go.Figure(data=[heat], layout=layout)
fig.show()

On peut remarquer une petite corrélation entre le "cahiertxt" et "noteelev", ce qui n'est pas trés surprenant car la plupart des établissements qui offrent la possibilité de consulter son cahier de texte en ligne (par exemple ENT) offre aussi la possibilité d'accéder aux notes de l'élève.

### Clustering

On peut maintenant faire du clustering avec l'algorithme de KMeans en essayant plusieurs paramètres pour l'algo. Nousallons laisser seulement les résultats avec les meilleurs paramètres. Nos essais précédents n'apparaîtront pas.

In [32]:
km_services = KMeans(
    n_clusters=4, 
    random_state=0,
    n_init=20,
    max_iter=3600,
    init='k-means++'
)

y_km_services = km_services.fit_predict(df_services)

resultat = pd.DataFrame(km_services.cluster_centers_, columns=km_services.feature_names_in_)

resultat

,noteelev,abselev,edtelevcls,cahiertxt,docresspeda,agdactuetab,platapp,autres,aucun
0,0.994048,0.934524,1.000000,1.000000,0.988095,0.991071,-1.110223e-16,1.0,0.0
1,0.997275,0.959128,0.991826,1.000000,0.975477,0.978202,-1.942890e-16,0.0,0.0
2,0.991667,0.975000,0.966667,0.991667,1.000000,0.958333,1.000000e+00,1.0,0.0
3,1.000000,1.000000,1.000000,1.000000,0.977528,0.955056,1.000000e+00,0.0,0.0


On va convertir en "oui" et "non" pour que ces résultats soient plus parlant. Nous allons essayer d'interprèter ces centres pour pouvoir comprendre le nombre de classes et les caractèristiques moyennes d'une classe pour pouvoir lui accorder une modalité

In [33]:
def change_value(row):
    if(row > 0.5):
        return 1
    else:
        return 0


for column in resultat.columns:
    resultat[column] = resultat[column].apply(change_value)
    resultat[column] = resultat[column].replace({1: 'oui', 0: 'non'})
    resultat[column] = resultat[column].astype(str)


resultat

,noteelev,abselev,edtelevcls,cahiertxt,docresspeda,agdactuetab,platapp,autres,aucun
0,oui,oui,oui,oui,oui,oui,non,oui,non
1,oui,oui,oui,oui,oui,oui,non,non,non
2,oui,oui,oui,oui,oui,oui,oui,oui,non
3,oui,oui,oui,oui,oui,oui,oui,non,non
